In [0]:
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
import pandas as pd

import logging
logging.getLogger().setLevel(logging.ERROR)

In [3]:
listings=pd.read_csv('/content/listings.csv')
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,...,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,2.016010e+13,1/4/2016,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,NaN,NaN,https://a1.muscache.com/ac/pictures/67560560/c...,NaN,956883,https://www.airbnb.com/users/show/956883,Maija,8/11/2011,"Seattle, Washington, United States","I am an artist, interior designer, and run a s...",within a few hours,96%,100%,f,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,...,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,$85.00,NaN,NaN,NaN,NaN,2,$5.00,1,365,4 weeks ago,t,14,41,71,346,1/4/2016,207,11/1/2011,1/2/2016,95.0,10.0,10.0,10.0,10.0,9.0,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,2.016010e+13,1/4/2016,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/im/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,5177328,https://www.airbnb.com/users/show/5177328,Andrea,2/21/2013,"Seattle, Washington, United States",Living east coast/left coast/overseas. Time i...,within an hour,98%,100%,t,https://a0.muscache.com/ac/users/5177328/profi...,https://a0.muscache.com/ac/users/5177328/profi...,Queen Anne,6.0,6.0,"['email', 'phone', 'facebook', 'linkedin', 're...",t,t,"7th Avenue West, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,...,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",NaN,$150.00,"$1,000.00","$3,000.00",$100.00,$40.00,1,$0.00,2,90,today,t,13,13,16,291,1/4/2016,43,8/19/2013,12/29/2015,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,2.016010e+13,1/4/2016,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,Our house is located just 5 short blocks to To...,A bus stop is just 2 blocks away. Easy bus a...,NaN,NaN,https://a2.muscache.co

In [0]:

#Converting into numeric data
listings['bedrooms'] = pd.to_numeric(listings['bedrooms'],errors='coerce')
listings['accommodates'] = pd.to_numeric(listings['accommodates'],errors='coerce')
listings['bathrooms'] = pd.to_numeric(listings['bathrooms'],errors='coerce')
listings['number_of_reviews']=pd.to_numeric(listings['number_of_reviews'],errors='coerce')
listings['beds']=pd.to_numeric(listings['beds'],errors='coerce')
listings['number_of_reviews']=pd.to_numeric(listings['number_of_reviews'],errors='coerce')
listings['host_listings_count']=pd.to_numeric(listings['host_listings_count'],errors='coerce')
listings['review_scores_rating']=pd.to_numeric(listings['review_scores_rating'],errors='coerce')
listings['zipcode']=pd.to_numeric(listings['zipcode'],errors='coerce')
listings['monthly_price'] = pd.to_numeric(listings['monthly_price'].apply(lambda x: str(x).replace('$', '').replace(',', '')),errors='coerce')
listings['price'] = pd.to_numeric(listings['price'].apply(lambda x: str(x).replace('$', '').replace(',', '')),errors='coerce')
listings['weekly_price'] = pd.to_numeric(listings['weekly_price'].apply(lambda x: str(x).replace('$', '').replace(',', '')),errors='coerce')
listings['security_deposit'] = pd.to_numeric(listings['security_deposit'].apply(lambda x: str(x).replace('$', '').replace(',', '')),errors='coerce')
listings['cleaning_fee'] = pd.to_numeric(listings['cleaning_fee'].apply(lambda x: str(x).replace('$', '').replace(',', '')),errors='coerce')
listings['extra_people'] = pd.to_numeric(listings['extra_people'].apply(lambda x: str(x).replace('$', '').replace(',', '')),errors='coerce')
listings['host_response_rate'] = pd.to_numeric(listings['host_response_rate'].apply(lambda x: str(x).replace('%', '')),errors='coerce')

In [0]:
# Data pre-processing
def preprocess(df):
    
    temp_df = df.copy()
    temp_df = temp_df.replace(
            {
            'host_has_profile_pic': {'t': True, 'f': False},
            'host_identity_verified': {'t': True, 'f': False},
            'instant_bookable': {'t': True, 'f': False},
            }
    )
        
    ## Recode property_type
    def recode_prop(value):
        if value not in ['House', 'Apartment','Condominium','Townhouse','Loft']:
            return 'other_prop_type'
        return value

    temp_df['property_type'] = temp_df['property_type'].apply(recode_prop)

    ## Recode bed_type
    def recode_bed(value):
        if value not in ['Real Bed']:
            return 'other_bed_type'
        return value

    temp_df['bed_type'] = temp_df['bed_type'].apply(recode_bed)
        
    #Calculate the bedroom and bathroom share per person. Lower the share, more the comfort.
    temp_df = temp_df.assign(
        bedroom_share = temp_df.bedrooms/temp_df.accommodates,
        bathroom_share = temp_df.bathrooms/temp_df.accommodates,
        
    )
    
    df=temp_df
    print("Pre-processing completed...")
    return df


In [6]:
#Preprocess the listings data
listings=preprocess(listings)

Pre-processing completed...


In [7]:
listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [0]:
df = listings[["host_response_rate", "host_acceptance_rate", "host_is_superhost","security_deposit","cleaning_fee","extra_people",
                   "host_listings_count", "zipcode", "property_type","room_type", "accommodates", "bathrooms", "bedrooms",
                   "beds", "price", "number_of_reviews", "review_scores_rating", "cancellation_policy", 
                   "reviews_per_month"]]

In [9]:
# Handling the missing values:
df.price=df.price.fillna(df.price.mean)
df.security_deposit=df.security_deposit.fillna(0)
df.reviews_per_month=df.reviews_per_month.fillna(0)
df.cleaning_fee=df.cleaning_fee.fillna(0)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [0]:
# drop NaN rows
df2=df.dropna(axis=0)

In [0]:
pd.options.mode.chained_assignment = None  # default='warn'
df2['host_response_rate'] = df2['host_response_rate'].astype(str)
df2['host_acceptance_rate'] = df2['host_acceptance_rate'].astype(str)
df2['price'] = df2['price'].astype(str)

# clean data
pd.options.mode.chained_assignment = None  # default='warn'
df2['host_acceptance_rate'] = df2['host_acceptance_rate'].str.replace("%", "").astype("float")
df2['price'] = df2['price'].str.replace("[$, ]", "").astype("float")
df2['host_response_rate'] = df2['host_response_rate'].str.replace("%", "").astype("float")

df2['superhost']=np.where(df2['host_is_superhost']=='t',1,0)
del df2['host_is_superhost']


# select non-numeric variables and create dummies
non_num_vars = df2.select_dtypes(include=['object']).columns
dummy_vars = pd.get_dummies(df2[non_num_vars])

# drop non-numeric variables from df2 and add the dummies
df3=df2.drop(non_num_vars,axis=1)
df3 = pd.merge(df3,dummy_vars, left_index=True, right_index=True)

#Droping the missing values

df3=df3.dropna()

In [0]:
target_variable = ["price"]
y = df3['price']
x = df3.drop('price', axis =1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df3.drop(target_variable, axis=1),df3.price,test_size = 0.20, random_state=1)
model = linear_model.LinearRegression()
model.fit(X_train, y_train)
y_test_pred=model.predict(X_test)
y_train_pred=model.predict(X_train)
rms_test=sqrt(mean_squared_error(y_test,y_test_pred))
mean_square_train=mean_squared_error(y_train, y_train_pred)
mean_square_test=mean_squared_error(y_test, y_test_pred)
r2_score_train=(r2_score(y_train, y_train_pred))
r2_score_test=r2_score(y_test, y_test_pred)
print('RMSE',rms_test)
print('r2 score train',r2_score_train)
print('r2 score test',r2_score_test)
print('Mean Square Error Train',mean_square_train)
print('Mean Square Error Test',mean_square_test)


RMSE 51.067863080437654
r2 score train 0.6137039537253899
r2 score test 0.6306236585589899
Mean Square Error Train 2523.0469694444787
Mean Square Error Test 2607.926639602327


In [14]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=500, 
                               criterion='mse', 
                               random_state=3, 
                               n_jobs=-1)
forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)
rms_test_rf=sqrt(mean_squared_error(y_test,y_test_pred))
mean_square_train=mean_squared_error(y_train, y_train_pred)
mean_square_test=mean_squared_error(y_test, y_test_pred)
r2_score_train=(r2_score(y_train, y_train_pred))
r2_score_test=r2_score(y_test, y_test_pred)
print('RMSE',rms_test_rf)
print('r2 score train',r2_score_train)
print('r2 score test',r2_score_test)
print('Mean Square Error Train',mean_square_train)
print('Mean Square Error Test',mean_square_test)

RMSE 46.75327242078409
r2 score train 0.948828717879159
r2 score test 0.6904022948687241
Mean Square Error Train 334.2191811763879
Mean Square Error Test 2185.8684820520502


In [15]:
import xgboost
#train XGBoost model
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
xgb.fit(X_train,y_train)
y_train_pred_xg = xgb.predict(X_train)
y_test_pred_xg = xgb.predict(X_test)
rms_test_xg=sqrt(mean_squared_error(y_test,y_test_pred_xg))
mean_square_train_xg=mean_squared_error(y_train, y_train_pred_xg)
mean_square_test_xg=mean_squared_error(y_test, y_test_pred_xg)
r2_score_train_xg=(r2_score(y_train, y_train_pred_xg))
r2_score_test_xg=r2_score(y_test, y_test_pred_xg)
print('RMSE',rms_test_xg)
print('r2 score train',r2_score_train_xg)
print('r2 score test',r2_score_test_xg)
print('Mean Square Error Train',mean_square_train_xg)
print('Mean Square Error Test',mean_square_test_xg)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[03:13:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE 46.46217976137158
r2 score train 0.9645256325518028
r2 score test 0.6942454938382172
Mean Square Error Train 231.69663822939432
Mean Square Error Test 2158.7341481780068


In [16]:
from sklearn.tree import DecisionTreeRegressor
dtr= DecisionTreeRegressor(max_depth=500)
dtr.fit(X_train,y_train)
y_train_pred_dt = dtr.predict(X_train)
y_test_pred_dt = dtr.predict(X_test)
rms_test_dt=sqrt(mean_squared_error(y_test,y_test_pred_dt))
mean_square_train_dt=mean_squared_error(y_train, y_train_pred_dt)
mean_square_test_dt=mean_squared_error(y_test, y_test_pred_dt)
r2_score_train_dt=(r2_score(y_train, y_train_pred_dt))
r2_score_test_dt=r2_score(y_test, y_test_pred_dt)
print('RMSE',rms_test_dt)
print('r2 score train',r2_score_train_dt)
print('r2 score test',r2_score_test_dt)
print('Mean Square Error Train',mean_square_train_dt)
print('Mean Square Error Test',mean_square_test_dt)

RMSE 60.43684350403069
r2 score train 0.9999774736702629
r2 score test 0.48265857779391563
Mean Square Error Train 0.1471280602636535
Mean Square Error Test 3652.6120527306966


In [0]:
#Pickling the best model i.e. Random Forest
import pickle 

random_forest_pkl_filename = r'price_predictor_model.pickle'
random_forest_model_pkl = open(random_forest_pkl_filename, 'wb')

# Save the trained model as a pickle string. 
pickle.dump(forest,random_forest_model_pkl) 

random_forest_model_pkl.close()

In [20]:
f_myfile = open('/content/price_predictor_model.pickle', 'rb')
# Load the pickled model 
random_forest_from_pickle = pickle.load(f_myfile) 
  
# Use the loaded pickled model to make predictions 
random_forest_from_pickle.predict(X_test) 

array([116.698     , 295.236     ,  69.318     , 116.588     ,
        98.262     , 142.9       ,  64.828     ,  73.936     ,
        99.318     ,  84.628     , 101.19      ,  76.962     ,
       125.998     , 238.172     ,  61.986     ,  99.702     ,
       145.474     , 113.996     , 144.118     , 118.206     ,
       146.526     ,  96.052     ,  65.822     ,  77.152     ,
       115.194     , 113.108     , 136.956     , 109.428     ,
       151.954     ,  94.318     ,  72.374     ,  55.312     ,
        47.05      , 166.684     ,  95.406     ,  97.752     ,
        51.83      ,  69.224     , 113.244     , 117.352     ,
        92.034     ,  53.376     ,  42.956     , 422.376     ,
       119.97      , 116.33      ,  69.564     ,  95.444     ,
       243.52      ,  63.326     , 212.432     ,  31.51      ,
       135.886     ,  65.354     , 109.35      , 309.636     ,
        63.37      , 104.698     , 101.764     ,  77.056     ,
        49.856     , 232.226     ,  99.2       , 136.87